In [1]:
import requests
import json
import os

def get_spotify_access_token(client_id, client_secret):
    auth_response = requests.post('https://accounts.spotify.com/api/token', {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret,
    })
    return auth_response.json().get('access_token')

def search_violin_tracks(access_token):
    headers = {'Authorization': f'Bearer {access_token}'}
    search_query = 'violin'
    search_url = f'https://api.spotify.com/v1/search?q={search_query}&type=track&market=US&limit=50'
    response = requests.get(search_url, headers=headers)
    return response.json()

def download_track_previews(tracks, download_folder):
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    for track in tracks['tracks']['items']:
        preview_url = track['preview_url']
        if preview_url:
            track_id = track['id']
            file_name = os.path.join(download_folder, f"{track_id}.mp3")
            with requests.get(preview_url, stream=True) as r:
                with open(file_name, 'wb') as f:
                    for chunk in r.iter_content(chunk_size=8192):
                        f.write(chunk)
            print(f"Downloaded preview: {file_name}")

client_id = '9ef132f793af4e49b3dfccf785092a4b'
client_secret = '67639ea1e6db42a388e6ec7d083644d8'

token = get_spotify_access_token(client_id, client_secret)
violin_tracks = search_violin_tracks(token)

# Specify the folder where you want to download the tracks
download_folder = 'violin_track_previews'
download_track_previews(violin_tracks, download_folder)


Downloaded preview: violin_track_previews/4se25yP3bMpaMAlcxvyJ5Q.mp3
Downloaded preview: violin_track_previews/4IrWyseKFBykjoTm67NnNU.mp3
Downloaded preview: violin_track_previews/7y9ygf5AHcHwoSlnhFvF6F.mp3
Downloaded preview: violin_track_previews/716c0Rl8bBbbVNVM3r3pUq.mp3
Downloaded preview: violin_track_previews/2skMRTMYIi2FYZgprAouKC.mp3
Downloaded preview: violin_track_previews/4g6q38qYGf4TOfeFcJHLc3.mp3
Downloaded preview: violin_track_previews/1YFA2vnDRM6NtZlh0U3kRq.mp3
Downloaded preview: violin_track_previews/77TMuFit7SUhof0gl8LSpE.mp3
Downloaded preview: violin_track_previews/4S1vsHOlmAzp8nu3Sh7fR9.mp3
Downloaded preview: violin_track_previews/2pcqjh978FXjLnRcv22x2h.mp3
Downloaded preview: violin_track_previews/0pVbzd0lxd7y8oQWTTRuOy.mp3
Downloaded preview: violin_track_previews/52oy0ug2tUPaJHEhbubZ5P.mp3
Downloaded preview: violin_track_previews/3JMDJrW3kuH8KJWA0hi3As.mp3
Downloaded preview: violin_track_previews/6UvgVtAhaIFmGuWpDr2OmT.mp3
Downloaded preview: violin_track_p

In [4]:
import requests
import csv
import os
import time

def get_spotify_access_token(client_id, client_secret):
    auth_response = requests.post('https://accounts.spotify.com/api/token', {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret,
    })
    return auth_response.json().get('access_token')

def search_violin_tracks(access_token, query, offset=0, limit=50):
    headers = {'Authorization': f'Bearer {access_token}'}
    search_url = f'https://api.spotify.com/v1/search?q={query}&type=track&market=US&limit={limit}&offset={offset}'
    response = requests.get(search_url, headers=headers)
    return response.json()

def download_track_previews(tracks, download_folder, metadata_file):
    with open(metadata_file, 'a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        for track in tracks['tracks']['items']:
            preview_url = track['preview_url']
            if preview_url:
                track_id = track['id']
                track_name = track['name']
                artist_name = track['artists'][0]['name']  # Taking the first artist
                album_name = track['album']['name']
                file_name = os.path.join(download_folder, f"{track_id}.mp3")
                writer.writerow([track_id, track_name, artist_name, album_name, preview_url])
                download_preview(preview_url, file_name)
                print(f"Downloaded preview: {file_name}")

def download_preview(url, file_name):
    with requests.get(url, stream=True) as r:
        with open(file_name, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)

def main():
    client_id = '9ef132f793af4e49b3dfccf785092a4b'
    client_secret = '67639ea1e6db42a388e6ec7d083644d8'
    download_folder = 'violin_track_previews'
    metadata_file = 'violin_track_metadata.csv'
    search_queries = ['violin instrumental', 'strings instrumental']

    token = get_spotify_access_token(client_id, client_secret)

    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    if not os.path.exists(metadata_file):
        with open(metadata_file, 'w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(['Track ID', 'Track Name', 'Artist Name', 'Album Name', 'Preview URL'])

    for query in search_queries:
        offset = 0
        while True:
            try:
                tracks = search_violin_tracks(token, query, offset)
                if not tracks['tracks']['items']:
                    break
                download_track_previews(tracks, download_folder, metadata_file)
                offset += 50
                time.sleep(1)  # Pause to respect rate limits
            except KeyError:
                break

if __name__ == "__main__":
    main()


Downloaded preview: violin_track_previews/3JMDJrW3kuH8KJWA0hi3As.mp3
Downloaded preview: violin_track_previews/6b38muLpPpwr8N4OWhYJQQ.mp3
Downloaded preview: violin_track_previews/4IrWyseKFBykjoTm67NnNU.mp3
Downloaded preview: violin_track_previews/0KJrH9rjvyuvfnVk0O9nOF.mp3
Downloaded preview: violin_track_previews/1YFA2vnDRM6NtZlh0U3kRq.mp3
Downloaded preview: violin_track_previews/2E4s14vBrzRue0w7HRcer7.mp3
Downloaded preview: violin_track_previews/2skMRTMYIi2FYZgprAouKC.mp3
Downloaded preview: violin_track_previews/2mf9NnHqn2myJjs9y0SYAv.mp3
Downloaded preview: violin_track_previews/4ymzuzQyU6QzEzLDW5TeuX.mp3
Downloaded preview: violin_track_previews/2uHVKpROvu6m23LBicwBf4.mp3
Downloaded preview: violin_track_previews/1ZKGS3HzJeVqvBSh3tSMbU.mp3
Downloaded preview: violin_track_previews/2ZMFddysIOuCWxYZNcMz1M.mp3
Downloaded preview: violin_track_previews/36KTjwL6CA94PTxkFVijdI.mp3
Downloaded preview: violin_track_previews/313Z6hsAGtLPE3u2KfKHdT.mp3
Downloaded preview: violin_track_p

In [1]:
import pandas as pd

In [2]:
pip install pydub

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from pydub import AudioSegment

def convert_audio_for_jukebox(input_path, output_path, target_sample_rate=44100):
    # Load the MP3 file
    audio = AudioSegment.from_file(input_path, format="mp3")

    # Convert to the target sample rate (Jukebox's requirement)
    audio = audio.set_frame_rate(target_sample_rate)

    # Export the converted file
    # You may choose to change the format (e.g., to WAV) depending on Jukebox's requirements
    audio.export(output_path, format="wav")

# Example usage
convert_audio_for_jukebox("data/violin_track_previews/inputfile.mp3", "path/to/your/outputfile.wav")
